# Validate Dataset

The paper references the Amazon Review 2014 dataset with some statistics. This is to verify the information is the same and ensure we use the same dataset to replicate.

In [7]:
import pandas as pd
from tiger.data.amazon import AmazonDataset

In [2]:
MIN_REVIEWS_PER_USER = 5
SEQUENCE_LENGTH = 20

In [3]:
beauty_ds = AmazonDataset("Beauty", 2014, MIN_REVIEWS_PER_USER, "../data")

In [4]:
sports_ds = AmazonDataset("Sports_and_Outdoors", 2014, MIN_REVIEWS_PER_USER, "../data")

In [5]:
toys_ds = AmazonDataset("Toys_and_Games", 2014, MIN_REVIEWS_PER_USER, "../data")

In [8]:
pd.DataFrame.from_records([beauty_ds.calculate_stats(), sports_ds.calculate_stats(), toys_ds.calculate_stats()])

,dataset,num_users,num_items,num_reviews,avg_reviews_per_user,median_reviews_per_user,mean_reviews_per_item,sparsity
0,Beauty,52374,121291,469771,8.969546,7.0,3.873090,0.999926
1,Sports_and_Outdoors,84368,194715,717464,8.503983,6.0,3.684688,0.999956
2,Toys_and_Games,58315,165371,525535,9.012004,6.0,3.177915,0.999946


# 2013 Amazon Data Processing (Temp)

In [159]:
import gzip
import json  # Replace simplejson with the built-in json module in Python 3

def parse(filename):
    with gzip.open(filename, 'rt', encoding='utf-8') as f:  # Open file in text mode with utf-8 encoding
        entry = {}
        for l in f:
            l = l.strip()
            colonPos = l.find(':')
            if colonPos == -1:
                yield entry
                entry = {}
                continue
            eName = l[:colonPos]
            rest = l[colonPos+2:]
            entry[eName] = rest
        yield entry  # Yield last entry if file ends without an empty line

rows = []
for e in parse("../data/raw/2013/Beauty.txt.gz"):
    rows.append(eval(json.dumps(e)))

In [160]:
test = pd.DataFrame.from_dict(rows)

In [164]:
test

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B00064C0IU,Oscar Eau de Toilette for Women by Oscar de La...,24.19,A1FWT811DSZLC8,Heidi M,0/0,3.0,1360368000,doesn't last,very light scent that doesn't last very long. ...
1,B00064C0IU,Oscar Eau de Toilette for Women by Oscar de La...,24.19,A1THE6V6O8ROD4,"Donna Mpaulin ""PURPLE RAVEN""",0/0,5.0,1358467200,Smells divine.,"This is my second bottle of sheer freesia, I j..."
2,B00064C0IU,Oscar Eau de Toilette for Women by Oscar de La...,24.19,A176IQ7MVD3N6T,M. Avila,0/0,1.0,1357084800,Very disappointed!,This perfume is just AWFUL! Smells nothing lik...
3,B00064C0IU,Oscar Eau de Toilette for Women by Oscar de La...,24.19,A34BDX4JVMG23Y,Kim M. Colt,0/0,5.0,1326240000,Nice gift,This was a gift for my sister. It wowed her. S...
4,B000K5JBZU,Optimum Care Anti-Breakage Therapy Moisture Re...,5.99,A3UWJXJI7S3T05,PloveJ,1/1,5.0,1351987200,TRULY MADE A DIFFERENCE!,I have been using this product for a couple ye...
...,...,...,...,...,...,...,...,...,...,...
252052,B00025X06E,Goldleaf Perfumed Body Cream 230ml,28.00,A3M174IC0VXOS2,Gail Cooke,1/1,5.0,1250467200,A SWEET SCENT,Since its founding in 1982 Thymes has won legi...
252053,B00025X06E,Goldleaf Perfumed Body Cream 230ml,28.00,A2SZ9BG00RYAHG,&#34;Janie&#34;,0/0,5.0,1262476800,Luscious!!!,I love love love this item!!! The scent is lik...
252054,B000BR64OS,Guerlain Vetiver Eau de Toilette,unknown,A13NEK0V8EMHPO,PA Fishing Maps,6/6,2.0,1321228800,This is the new formula- NOT vintage as pictured,Received the order very quickly but what arriv...
252055,B000BR64OS,Guerlain Vetiver Eau de Toilette,unknown,AAX3RFP8NWI1U,Terry Grabow,0/1,5.0,1329177600,Great value for the Scents!!,Have been using this cologne since at least 19...


In [166]:
rc = test[['review/userId', 'product/productId']].groupby('review/userId').count().reset_index().rename({'product/productId': 'count'}, axis=1)
rc = rc.pipe(filter_less_than_k_reviews, k=5, col='count')

In [169]:
rc.shape

(4327, 2)

In [171]:
test[test['review/userId'].isin(rc['review/userId'].tolist())]['product/productId'].unique().shape

(10248,)

In [172]:
rc['count'].mean()

np.float64(10.652646175179108)